In [1]:
#INIZIALIZZAIZIONE SESSION E AVVIO SPARK
import pyspark as pys 
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
import pyspark.sql.functions as fn
#Spark session
spark = SparkSession.builder.getOrCreate()

#Spark context
sc = spark.sparkContext

#DF sporco di esempio
dirty_data = spark.createDataFrame([
(1,'Porsche','Boxster S','Turbo',2.5,4,22,None)
, (2,'Aston Martin','Vanquish','Aspirated',6.0,12,16,None)
, (3,'Porsche','911 Carrera 4S Cabriolet','Turbo',3.0,6,24,None)
, (3,'General Motors','SPARK ACTIV','Aspirated',1.4,None,32,None)
, (5,'BMW','COOPER S HARDTOP 2 DOOR','Turbo',2.0,4,26,None)
, (6,'BMW','330i','Turbo',2.0,None,27,None)
, (7,'BMW','440i Coupe','Turbo',3.0,6,23,None)
, (8,'BMW','440i Coupe','Turbo',3.0,6,23,None)
, (9,'Mercedes-Benz',None,None,None,None,27,None)
, (10,'Mercedes-Benz','CLS 550','Turbo',4.7,8,21,79231)
, (11,'Volkswagen','GTI','Turbo',2.0,4,None,None)
, (12,'Ford Motor Company','FUSION AWD','Turbo',2.7,6,20,None)
, (13,'Nissan','Q50 AWD RED SPORT','Turbo',3.0,6,22,None)
, (14,'Nissan','Q70 AWD','Aspirated',5.6,8,18,None)
, (15,'Kia','Stinger RWD','Turbo',2.0,4,25,None)
, (16,'Toyota','CAMRY HYBRID LE','Aspirated',2.5,4,46,None)
, (16,'Toyota','CAMRY HYBRID LE','Aspirated',2.5,4,46,None)
, (18,'FCA US LLC','300','Aspirated',3.6,6,23,None)
, (19,'Hyundai','G80 AWD','Turbo',3.3,6,20,None)
, (20,'Hyundai','G80 AWD','Turbo',3.3,6,20,None)
, (21,'BMW','X5 M','Turbo',4.4,8,18,121231)
, (22,'GE','K1500 SUBURBAN 4WD','Aspirated',5.3,8,18,None)
], ['Id','Manufacturer','Model','EngineType','Displacement',
'Cylinders','FuelEconomy','MSRP'])

dirty_data.show()


21/07/12 13:36:19 WARN Utils: Your hostname, lorenzo-X555UJ resolves to a loopback address: 127.0.1.1; using 192.168.43.245 instead (on interface wlp3s0)
21/07/12 13:36:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/12 13:36:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
+---+------------------+--------------------+----------+------------+---------+-----------+-----+
| Id|      Manufacturer|               Model|EngineType|Displacement|Cylinders|FuelEconomy| MSRP|
+---+------------------+--------------------+----------+------------+---------+-----------+-----+
|  1|           Porsche|           Boxster S|     Turbo|         2.5|        4|         22| null|
|  2|      As

In [6]:
#Elimino duplicati
numEl = dirty_data.count()
numElDist = dirty_data.distinct().count()
if numEl != numElDist:
    full_removed = dirty_data.drop_duplicates()

print("Count originale:\n" + str(numEl))

print("\nDopo eliminazione righe dupliate:")
print(full_removed.count())

#Potrebbero esserci righe duplicate con ID diverso

#Mostra le colonne duplicate senza ID
(
    full_removed.
    groupBy([col for col in full_removed.columns if col != "Id"])
    .count()
    .filter("count > 1")
    .show()
)

#Controllo duplicati su subset di colonne
id_removed=full_removed.dropDuplicates(
    subset = [col for col in full_removed.columns if col != "Id"]
)
id_removed.count()


Count originale:
22

Dopo eliminazione righe dupliate:
21
+------------+----------+----------+------------+---------+-----------+----+-----+
|Manufacturer|     Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|count|
+------------+----------+----------+------------+---------+-----------+----+-----+
|         BMW|440i Coupe|     Turbo|         3.0|        6|         23|null|    2|
|     Hyundai|   G80 AWD|     Turbo|         3.3|        6|         20|null|    2|
+------------+----------+----------+------------+---------+-----------+----+-----+



19

In [7]:
#ID duplicati (riga diversa)
import pyspark.sql.functions as fn
id_removed.agg(
    fn.count("Id").alias("Count ID"),
    fn.countDistinct("Id").alias("CountDistinct ID")
).show()

#Abbiamo 2 volte lo stesso ID
#Vediamo quale ID ricorre 2 volte
(
    id_removed.groupBy("Id").count().filter("count > 1").show()
)

#ID = 3 ricorre 2 volte
#NB id_removed contiene ancora il campo ID
#Creiamo un nuovo DF con un ID incrementato automaticamente
new_id =(
    id_removed.select([fn.monotonically_increasing_id().alias("Id")] +
    [col for col in id_removed.columns if col != "Id"])
)

new_id.show()

+--------+----------------+
|Count ID|CountDistinct ID|
+--------+----------------+
|      19|              18|
+--------+----------------+

+---+-----+
| Id|count|
+---+-----+
|  3|    2|
+---+-----+

+-------------+------------------+--------------------+----------+------------+---------+-----------+------+
|           Id|      Manufacturer|               Model|EngineType|Displacement|Cylinders|FuelEconomy|  MSRP|
+-------------+------------------+--------------------+----------+------------+---------+-----------+------+
|   8589934592|    General Motors|         SPARK ACTIV| Aspirated|         1.4|     null|         32|  null|
| 188978561024|     Mercedes-Benz|             CLS 550|     Turbo|         4.7|        8|         21| 79231|
| 197568495616|     Mercedes-Benz|                null|      null|        null|     null|         27|  null|
| 206158430208|Ford Motor Company|          FUSION AWD|     Turbo|         2.7|        6|         20|  null|
| 438086664192|               BMW|C

In [8]:
#Dati mancanti

#Lavoriamo sul RDD
#Vediamo in ogni riga quando dati mancano
(
spark.createDataFrame(
    new_id
    .rdd
    .map(
        lambda row: (
            row['Id']
            , sum([c == None for c in row])
            )
        )
        .filter(lambda el: el[1] > 1)
        .collect()
    ,['Id', 'CountMissing']
    )
    .orderBy('CountMissing', ascending=False)
    .show()
)

#Recuperiamo le righe che hanno attrivuti mancanti
new_id.where("Id = 197568495616").show()

#dropna(thresh=n) n è il numero di elementi mancanti sufficienti ad eliminare la riga
merc_out=new_id.dropna(thresh=4)
#Verifico contanto le righe
new_id.count() #19
merc_out.count() #18


#Valori mancanti per colonna
for k,v in sorted(
    merc_out.agg(*[
        #Calcolo la percentuale di mancanze per colonna
        (1-(fn.count(c)/fn.count("*")))
            .alias(c+"_miss")
        for c in merc_out.columns
        ])
    .collect()[0]
    .asDict()
    .items()
,key=lambda el: el[1]
,reverse=True

):
    print(k,v)
#MSRP_miss = 0.8 -> eliminiamo la colonna
no_MSRP = merc_out.select([col for col in merc_out.columns if col != "MSRP"])

no_MSRP.show()

#Altro metodo che dovrebbe funzionare (fatto da me)
d =merc_out.agg(*[
        #Calcolo la percentuale di mancanze per colonna
        (1-(fn.count(c)/fn.count("*")))
            .alias(c+"_miss")
        for c in merc_out.columns
        ]).collect()[0].asDict().items()
for k,v in sorted(d):
    print(k,v)

+------------+------------+
|          Id|CountMissing|
+------------+------------+
|197568495616|           5|
|  8589934592|           2|
|721554505728|           2|
|919123001344|           2|
+------------+------------+

+------------+-------------+-----+----------+------------+---------+-----------+----+
|          Id| Manufacturer|Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|
+------------+-------------+-----+----------+------------+---------+-----------+----+
|197568495616|Mercedes-Benz| null|      null|        null|     null|         27|null|
+------------+-------------+-----+----------+------------+---------+-----------+----+

MSRP_miss 0.8888888888888888
Cylinders_miss 0.11111111111111116
FuelEconomy_miss 0.05555555555555558
Id_miss 0.0
Manufacturer_miss 0.0
Model_miss 0.0
EngineType_miss 0.0
Displacement_miss 0.0
+-------------+------------------+--------------------+----------+------------+---------+-----------+
|           Id|      Manufacturer|               M

In [9]:
d =merc_out.agg(*[
        #Calcolo la percentuale di mancanze per colonna
        (1-(fn.count(c)/fn.count("*")))
            .alias(c+"_miss")
        for c in merc_out.columns
        ]).collect()[0].asDict().items()
for k,v in sorted(d):
    print(k,v)


Cylinders_miss 0.11111111111111116
Displacement_miss 0.0
EngineType_miss 0.0
FuelEconomy_miss 0.05555555555555558
Id_miss 0.0
MSRP_miss 0.8888888888888888
Manufacturer_miss 0.0
Model_miss 0.0


In [10]:
#Sostiuire i valori nulli

#Creiamo un dizionario con i moltiplicatori

multipliers = (
    no_MSRP.agg(
        fn.mean(#media
            fn.col("FuelEconomy") / 
            (
                fn.col("Displacement")*fn.col("Cylinders")
            )
        ).alias("FuelEconomy")
        , fn.mean(
            fn.col("Cylinders") / fn.col("Displacement")
        ).alias("Cylinders")
    )
).toPandas().to_dict("records")[0]


print(multipliers)
#Inseriamo i valori usando i moltiplicatori
#Prima di tuto facciamo in modo che tutti i valori rispettino la logica (la formula scelta prima), poi rimpiamo i null con i moltiplicatori

#Invece usiamo la formula inversa per ripristinare il valore originale

#RIASSUNTO
#Tutta la colonna -> moltiplicatori
#Inserisco moltiplicatore calcolati prima
#moltiplicatori -> valori originali

#Il metodo(nomeCol,col) creano o sostiuisce una colonna con i valori specificati da col
imputed = (
    no_MSRP
    #Calcolco i moltiplicatori
    .withColumn('FuelEconomy', fn.col('FuelEconomy') / fn.col('Displacement') / fn.col('Cylinders'))
    .withColumn('Cylinders', fn.col('Cylinders') / fn.col('Displacement'))
    #Sostituisco i null
    .fillna(multipliers)
    #Inverto e torno a valori iniziali
    .withColumn('Cylinders', (fn.col('Cylinders') *
    fn.col('Displacement')).cast('integer'))
    .withColumn('FuelEconomy', fn.col('FuelEconomy') * fn.col('Displacement') *
    fn.col('Cylinders'))
)

imputed.show()

#Non ci sono più valori nulli

{'FuelEconomy': 1.4957485048359973, 'Cylinders': 1.8353365984789105}
+-------------+------------------+--------------------+----------+------------+---------+------------------+
|           Id|      Manufacturer|               Model|EngineType|Displacement|Cylinders|       FuelEconomy|
+-------------+------------------+--------------------+----------+------------+---------+------------------+
|   8589934592|    General Motors|         SPARK ACTIV| Aspirated|         1.4|        2| 4.188095813540793|
| 188978561024|     Mercedes-Benz|             CLS 550|     Turbo|         4.7|        8|              21.0|
| 206158430208|Ford Motor Company|          FUSION AWD|     Turbo|         2.7|        5|16.666666666666668|
| 438086664192|               BMW|COOPER S HARDTOP ...|     Turbo|         2.0|        4|              26.0|
| 523986010112|      Aston Martin|            Vanquish| Aspirated|         6.0|       12|              16.0|
| 721554505728|        Volkswagen|                 GTI|    

In [12]:
#Valori anomali (outliers)

#Calcoliamo il range in cui i valori non sono anomali

#Colonne con valori numerici
features = ['Displacement', 'Cylinders', 'FuelEconomy']
quantiles = [0.25,0.75]

cut_off_point=[]

for feauture in features:
    #approxQuantile(colonna,quantiles,precisione (0 dispendiosa))
    quants = imputed.approxQuantile(feauture,quantiles,0.05)
    #quants è una lista, 1 e 3 quartile
    #interquantile range
    IQR = quants[1] - quants[0]
    cut_off_point.append( (feauture, [
        quants[0] - 1.5 * IQR,
        quants[1] + 1.5 * IQR
    ]))
cut_off_point=dict(cut_off_point)
print(cut_off_point)

#Marchiamo gli anomali
outliers=imputed.select(["Id"]+[
    (
        (imputed[f] < cut_off_point[f][0]) |
        (imputed[f]>cut_off_point[f][1])
    ).alias(f + "_o") for f in features
])
outliers.show()

#2 outlier nella colonna FuelEconomy

#Recupero gli outlier
with_outlier_flag = imputed.join(outliers,on="Id")

#Mostro i record con anomalie in FuelEconomy
(
    with_outlier_flag.filter("FuelEconomy_o")
    .select("Id",'Manufacturer', 'Model', 'FuelEconomy')
    .show()
)

#Gli outlier possono rendere inaffidabile l'algoritmo di ML
#Vanno eliminati

#Elimino gli outliers
no_outliers = (
    with_outlier_flag.filter("!FuelEconomy_o").select(imputed.columns)
)

no_outliers.show()

{'Displacement': [-0.40000000000000036, 6.0], 'Cylinders': [1.0, 9.0], 'FuelEconomy': [5.5, 33.5]}
+-------------+--------------+-----------+-------------+
|           Id|Displacement_o|Cylinders_o|FuelEconomy_o|
+-------------+--------------+-----------+-------------+
|   8589934592|         false|      false|         true|
| 188978561024|         false|      false|        false|
| 206158430208|         false|      false|        false|
| 438086664192|         false|      false|        false|
| 523986010112|         false|       true|        false|
| 721554505728|         false|      false|        false|
| 764504178688|         false|      false|        false|
| 919123001344|         false|      false|        false|
| 944892805120|         false|      false|        false|
| 970662608896|         false|      false|        false|
|1030792151040|         false|      false|        false|
|1039382085632|         false|      false|        false|
|1116691496960|         false|      false|    

In [13]:
#Statistica descrittiva

#Feautures è una lista di colonne con valori numerici
descriptive_stats = no_outliers.describe(features)
descriptive_stats.show()

#Calcolo su valori -aggregati
(
    no_outliers
    .select(features)
    .groupBy("Cylinders") #Raggruppo per numero di cilindri
    .agg(*[
        fn.count('*').alias("Count")
        , fn.mean("FuelEconomy").alias("MPG_avg")
        , fn.mean("Displacement").alias("Disp_avg")
        , fn.stddev("FuelEconomy").alias("MPG_stddev")
        , fn.stddev("Displacement").alias("Disp_stddev")
    ])
    .orderBy("Cylinders")
).show()

#Notiamo delle stranezze
#Il modo usato per sostituire i valori nulli era inaccurato!!!

#Se è presente un solo record alla stddev=null



+-------+-----------------+-----------------+------------------+
|summary|     Displacement|        Cylinders|       FuelEconomy|
+-------+-----------------+-----------------+------------------+
|  count|               16|               16|                16|
|   mean|          3.44375|            6.125|19.600446608398165|
| stddev|1.354975399530683|2.276693508870558| 4.666647767373751|
|    min|              2.0|                3| 8.974491029015983|
|    max|              6.0|               12|              26.0|
+-------+-----------------+-----------------+------------------+

+---------+-----+------------------+------------------+------------------+-------------------+
|Cylinders|Count|           MPG_avg|          Disp_avg|        MPG_stddev|        Disp_stddev|
+---------+-----+------------------+------------------+------------------+-------------------+
|        3|    1| 8.974491029015983|               2.0|              null|               null|
|        4|    4|21.24149700967199

In [14]:
#Correlazioni

#coefficiente di correlazione di Pearson
no_outliers.corr("Cylinders","Displacement")

#Matrice di correlazione
n_features = len(features)
corr = []

for i in range (0,n_features):
    temp = [None] * i

    for j in range(i,n_features):
        temp.append(no_outliers.corr(features[i],features[j]))
    corr.append([features[i]]+temp)
correlations = spark.createDataFrame(corr,["Colums"]+features)
correlations.show()

+------------+------------+------------------+--------------------+
|      Colums|Displacement|         Cylinders|         FuelEconomy|
+------------+------------+------------------+--------------------+
|Displacement|         1.0|0.9381829964408113|-0.10757908872387649|
|   Cylinders|        null|               1.0|-0.04218546545035...|
| FuelEconomy|        null|              null|                 1.0|
+------------+------------+------------------+--------------------+



In [34]:
#ISTOGRAMMI
#Metodo con calcolo parallelo e distrbuito

histogram_MPG =(
    no_outliers
    .select('FuelEconomy')
    .rdd #DF non ha metodo per istogrammi
    .flatMap(lambda record: record)
    .histogram(5)
)

print(histogram_MPG)

#Creo DF di appoggio
(
    spark.createDataFrame(
        [(bins,counts) for bins, counts in zip(
        histogram_MPG[0],histogram_MPG[1])
        ]
        , ["bins","counts"]
    )
    .registerTempTable('histogram_MPG')
)



([8.974491029015983, 12.379592823212786, 15.78469461740959, 19.189796411606395, 22.594898205803197, 26.0], [2, 0, 5, 4, 5])
UsageError: Line magic function `%%sql` not found.


In [47]:
#PARTE SUI GRAFICI MANCANTE
#Problemi con codice magic

SyntaxError: invalid syntax (1853804402.py, line 3)

UsageError: Cell magic `%%local` not found.
